# Report #1 - 15.04.2021

## Previous research

### GANs
GANs where introduced in 2014 (Generative Adversarial Networks - Godfellow et al. 2014) and since then, a lot of research around image GANs has been done (for example deep convolutionl GAN) and it's now possible to generate very high quality images. However the same can't be said for text GANs (usually based on LSTM blocks) which received less attention, probably because of the inherent difficulties in training GANs (mode collapse, non-convergence and instability) and the availibility of other architectures like the trasnformer. 


### The Transformer
In the past years transformers ("Attention is all you need - Vaswani et al, 2017") have shown excellent performance in NLP tasks, in particular language trasnlation, thanks to their multi-head-self-attention component. They are now the default choice for NLP tasks and present in state of the art text nerual networks like BERT ("BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding" - Devlin et al. 2018) and the GPT family (OpenaAI).


### GAN_EVO
So far the GAN_EVO project focused on implementing evolutionary features in a population of image GANs to tackle common issues that can arise during training, in particular it improve the diversity of their output and avoid mode collapse. 
A translation to PyTorch of the Fréchet Inception Distance metric is also part of the repository.

### ScratchGAN
ScratchGAN ("Training Language GANs from Scratch", d'Automne et al. 2020) is the first text GAN architectures that doesn't require any MLE pretraining, and provide a set of useful tricks to avoid common issues during training. In particular the authors recommand to use large mini-bacthes, dense rewards (at word-level), big datasets and reinforcement learning for training. As expected for text GANs, the architecture is based on LSTM blocks.


## Research and results

### Research project
The goal of this research project is to show that high quality text generation through text GANs is possible and consequently it should be considered as a potential threat to public information. 

### First attempt
To achieve state of the art quality in text generation through GAN, we are investigating the possibility of replacing the LSTM module in an existing text GAN with a transformer layer. 

We discovered and decided to use the repository TextGAN-PyTorch (https://github.com/williamSYSU/TextGAN-PyTorch) as our starting point for experimentation with text GANs. Once we understood its structure, we picked the DP-GAN (Diversity-Promoting Generative Adversarial Network for Generating Informative and Diversified Text, Xu et al. 2018) implementation as our candidate for the transformer layer implementation; this experiment take the name of SADPGAN where the "SA" stands for Self Attention. 


The main challenge consisted in harmonizing the newly introduced self attention layer with the rest of the repository: forward functions for trasnformer and lstm take different parameters and thus a general refactoring would be necessary. Since we're actively experimenting and code is quickly changing, as a workaround we created a temporary copy adapted for self attention for each part of the repository. Below are reported the structure and forward function of the LSTM-based generator and respectively of our version implementing a trasformer layer.

In [ ]:
#LSTMGenerator, orginal code
def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, padding_idx, gpu=False):
    ...
    self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
    self.lstm2out = nn.Linear(hidden_dim, vocab_size)
    self.softmax = nn.LogSoftmax(dim=-1)
    ...

def forward(self, inp, hidden, need_hidden=False):    
    emb = self.embeddings(inp)  # batch_size * len * embedding_dim
    if len(inp.size()) == 1:
        emb = emb.unsqueeze(1)  # batch_size * 1 * embedding_dim
    out, hidden = self.lstm(emb, hidden)  # out: batch_size * seq_len * hidden_dim
    out = out.contiguous().view(-1, self.hidden_dim)
    out = self.lstm2out(out)  # (batch_size * seq_len) * vocab_size
    pred = self.softmax(out)
    if need_hidden:
        return pred, hidden
    else:
        return pred

#TransformerGenerator, our experiment
def __init__(self, embedding_dim, hidden_dim, vocab_size, max_seq_len, padding_idx, num_heads=4, nlayers=4, dropout=0.5, gpu=False):
    ...
    self.encoder = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
    self.pos_encoder = PositionalEncoding(embedding_dim, dropout)
    encoder_layers = TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim, dropout)
    self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
    self.decoder = nn.Linear(embedding_dim, vocab_size)
    self.softmax = nn.LogSoftmax(dim=-1)  
    ...  

def forward(self, inp, src_mask=None):
        src = self.encoder(inp) * math.sqrt(self.embedding_dim)
        if len(inp.size()) == 1:
            src = src.unsqueeze(1) 
        src = self.pos_encoder(src)
        #output = self.transformer_encoder(src, src_mask)
        output = self.transformer_encoder(src)
        output = output.contiguous().view(-1, self.hidden_dim)
        output = self.decoder(output)
        pred = self.softmax(output)
        return pred

As it can be seen, we hevely reley on the original code of the LSTMGenerator, but at the same time we replace the LSTM block with the encoder layer of the Transformer, without forgetting the positional encoders. For the sake of simplicity, in this first attempt we deliberatly omitted the src_mask in the forward function. These modifications were inspired by the PyTorch tutorial "SEQUENCE-TO-SEQUENCE MODELING WITH NN.TRANSFORMER AND TORCHTEXT" (https://pytorch.org/tutorials/beginner/transformer_tutorial.html), and by the architecture of TransGAN (Two Transformers Can Make One Strong GAN, Jiang et al. 2021), a very recent GAN replacing convolutional layers with attention ones to generate images.

![](transgan.png)

*TransGAN architecture only use the encoder layer of the transformer*

### Results
We trained both DPGAN and SADPGAN on the same machine, 120 epoch of MLE pretraining and 200 adversarial rounds with neural networks of similar size. Unfortunately we observed that no learning is occuring with the actual implementation of SADPGAN: both for quality and diversity, the network doesn't improve at all during pretraining. Without surprise the abscence of improvement persisted also during adversarial training.

| <img src="G_MLE_NLL_div.png"/> | <img src="G_MLE_NLL_gen.png"/> |
|--------------------------------|--------------------------------|

Cleary this first implementation of SADPGAN is not delivering the expected results. However we reamain confident in our idea of replacing the LSTM block with a transformer layer, and believe that the total abscence of learning is due to poor implementation rather than a conceptual mistake. 

### Second attempt


In our second attempt we implement the attention mask. While doing so we discovered a conceptual error in the transition from LSTM to transformer blocks: contrary to the forward pass of LSTM that need as input a 1D tensor of size batch_size, transformers need a 2D tensor \[max_seq_len, batch_size\]. Second mistake was to create a view of dimension \[batch_size \* seq_len, hidden_dim\]: now at then end of the forward pass the tenosr is of size \[max_seq_len * batch_size, vocab_size\] where vocab_size is a probaility density over all words of the given vocabulary. 

Multinomial sampling revealed to be tricky since we're expected to fill chuncks of batch_size items but the first dimension of our tensor is batch_size\*max_seq_len. Waiting for a satisfacory solution to this issue for now we're expanding the tensor back to \[max_seq_len \* batch_size, vocab_size\] and then discard the first dimension. See the code below for clarity
 

In [ ]:
def forward(self, src, src_mask=None):
    #src: [max_seq_len, batch_size]
    src1 = self.encoder(src) * math.sqrt(self.embedding_dim) #src1: [max_seq_len, batch_size, embedding_dim]
    src2 = self.pos_encoder(src1) #src2: [max_seq_len, batch_size, embedding_dim]
    output1 = self.transformer_encoder(src2, src_mask) #output1: [max_seq_len, batch_size, embedding_dim]
    output2 = self.decoder(output1) #output2: [max_seq_len, batch_size, vocab_size]
    output3 = output2.contiguous().view(-1, self.vocab_size)  # [max_seq_len * batch_size, vocab_size]
    pred = self.softmax(output) # [max_seq_len * batch_size, vocab_size]
    return pred


def sample(self, num_samples, batch_size, start_letter=cfg.start_letter):
    """
    Samples the network and returns num_samples samples of length max_seq_len.
    :return samples: num_samples * max_seq_length (a sampled sequence in each row)
    """
    num_batch = num_samples // batch_size + 1 if num_samples != batch_size else 1
    samples = torch.zeros(num_batch * batch_size, self.max_seq_len).long()

     # Generate sentences with multinomial sampling strategy
    for b in range(num_batch):
        #inp = torch.LongTensor([start_letter] * batch_size)
        inp = torch.LongTensor([start_letter] * self.max_seq_len)  # [max_seq_len]
        inp = inp.unsqueeze(1).expand(self.max_seq_len, batch_size)  # [max_seq_len, batch_size]

        if self.gpu:
            inp = inp.cuda()

        for i in range(self.max_seq_len):
            out = self.forward(inp, self.generate_square_subsequent_mask(self.max_seq_len))  
            # [max_seq_len * batch_size, vocab_size]
            
            #Expand to 3 dimesnion and then drop the first one of size max_seq_len
            pred = torch.reshape(out, (self.max_seq_len, batch_size, self.vocab_size)) 
            # [max_seq_len, batch_size, vocab_size]
            pred = pred[i, :, :]        # [batch_size, vocab_size]
            
            next_token = torch.multinomial(torch.exp(pred), 1)  # [batch_size, 1] (sampling from each row)
            samples[b * batch_size : (b + 1) * batch_size, i] = next_token.view(-1)
            inp = next_token.view(-1).unsqueeze(0).expand(self.max_seq_len, batch_size) # [max_seq_len, batch_size]
    samples = samples[:num_samples]
    return samples

### Results
With these modifications, masking and multihead attention operation are correctly performed and the tensor dimensions are consistent with what is expected during a forward pass of a transformer. However this didn't result in an improvement of the learning proces: the model performance is constant over time as in the first attempt. 

## Conclusions and future work
Our priority is to understand what went wrong with the training of SADPGAN. A potential source of problem could be a wrong implementation of the characteristic word-level feedback of DPGAN in SADPGAN. But our main lead is now regards the sampling process where we brutally discrard one dimension: we should understand how to correctly sample over a  distribution of size \[max_seq_len * batch_size, vocab_size\] instead of \[batch_size, vocab_size\].
In addition we should not exclude other issues related to the architecture of our model, for example the decision of relying only on the encoder part of the Transformer. 

We'll investigate all these potetntial issues and once addressed we expect our model to be able to imporve over training. The rate of learning will guide our next steps: in case of better results than the original version we'll move on other exisiting GANs to see if we're able to replicate this result. If not we'll further investigate how to imporve the implementation of the self attention layer until we achieve a result at least comparable to the original version. 

## Appendix

### A. Parameters for DPGAN and SADPGAN training

Here is the list of full parameters for both experiments:

| DPGAN > training arguments: >>> if_test: 0 >>> run_model: dpgan >>> k_label: 2 >>> dataset: oracle >>> model_type: vanilla >>> loss_type: rsgan >>> mu_type: ragan >>> eval_type: Ra >>> d_type: Ra >>> if_real_data: 0 >>> sa: False >>> cuda: 1 >>> device: 0 >>> devices: 0 >>> shuffle: 0 >>> gen_init: normal >>> dis_init: uniform >>> n_parent: 1 >>> eval_b_num: 8 >>> lambda_fq: 1.0 >>> lambda_fd: 0.0 >>> d_out_mean: True >>> freeze_dis: False >>> freeze_clas: False >>> use_all_real_fake: False >>> use_population: False >>> samples_num: 1000 >>> vocab_size: 5000 >>> mle_epoch: 120 >>> clas_pre_epoch: 10 >>> adv_epoch: 200 >>> inter_epoch: 15 >>> batch_size: 64 >>> max_seq_len: 20 >>> start_letter: 1 >>> padding_idx: 0 >>> gen_lr: 0.01 >>> gen_adv_lr: 0.0001 >>> dis_lr: 0.01 >>> clip_norm: 5.0 >>> pre_log_step: 10 >>> adv_log_step: 1 >>> train_data: dataset/oracle.txt >>> test_data: dataset/testdata/oracle_test.txt >>> temp_adpt: exp >>> evo_temp_step: 1 >>> temperature: 1 >>> ora_pretrain: 1 >>> gen_pretrain: 0 >>> dis_pretrain: 0 >>> adv_g_step: 1 >>> rollout_num: 16 >>> gen_embed_dim: 32 >>> gen_hidden_dim: 32 >>> goal_size: 16 >>> step_size: 4 >>> mem_slots: 1 >>> num_heads: 4 >>> head_size: 64 >>> gen_nlayers: 3 >>> gen_num_heads: 4 >>> dropout: 0.5 >>> d_step: 5 >>> d_epoch: 3 >>> adv_d_step: 4 >>> adv_d_epoch: 2 >>> dis_embed_dim: 64 >>> dis_hidden_dim: 64 >>> num_rep: 64 >>> dis_nlayers: 2 >>> dis_num_heads: 4 >>> use_nll_oracle: 1 >>> use_nll_gen: 1 >>> use_nll_div: 1 >>> use_bleu: 1 >>> use_self_bleu: 0 >>> use_clas_acc: True >>> use_ppl: 0 >>> log_file: log/log_0413_0931_30.txt >>> save_root: save/20210413/oracle/dpgan_vanilla_dt-Ra_lt-rsgan_mt-ra_et-Ra_sl20_temp1_lfd0.0_T0413_0931_30/ >>> signal_file: run_signal.txt >>> tips: DPGAN experiments | SADPGAN > training arguments: >>> if_test: 0 >>> run_model: sa_dpgan >>> k_label: 2 >>> dataset: oracle >>> model_type: pineapple >>> loss_type: rsgan >>> mu_type: ragan >>> eval_type: Ra >>> d_type: Ra >>> if_real_data: 0 >>> sa: 1 >>> cuda: 1 >>> device: 0 >>> devices: 0 >>> shuffle: 0 >>> gen_init: normal >>> dis_init: uniform >>> n_parent: 1 >>> eval_b_num: 8 >>> lambda_fq: 1.0 >>> lambda_fd: 0.0 >>> d_out_mean: True >>> freeze_dis: False >>> freeze_clas: False >>> use_all_real_fake: False >>> use_population: False >>> samples_num: 10000 >>> vocab_size: 5000 >>> mle_epoch: 120 >>> clas_pre_epoch: 10 >>> adv_epoch: 200 >>> inter_epoch: 15 >>> batch_size: 64 >>> max_seq_len: 20 >>> start_letter: 1 >>> padding_idx: 0 >>> gen_lr: 0.01 >>> gen_adv_lr: 0.0001 >>> dis_lr: 0.01 >>> clip_norm: 5.0 >>> pre_log_step: 10 >>> adv_log_step: 1 >>> train_data: dataset/oracle.txt >>> test_data: dataset/testdata/oracle_test.txt >>> temp_adpt: exp >>> evo_temp_step: 1 >>> temperature: 1 >>> ora_pretrain: 0 >>> gen_pretrain: 0 >>> dis_pretrain: 0 >>> adv_g_step: 1 >>> rollout_num: 16 >>> gen_embed_dim: 32 >>> gen_hidden_dim: 32 >>> goal_size: 16 >>> step_size: 4 >>> mem_slots: 1 >>> num_heads: 4 >>> head_size: 64 >>> gen_nlayers: 4 >>> gen_num_heads: 4 >>> dropout: 0.5 >>> d_step: 5 >>> d_epoch: 3 >>> adv_d_step: 4 >>> adv_d_epoch: 2 >>> dis_embed_dim: 64 >>> dis_hidden_dim: 64 >>> num_rep: 64 >>> dis_nlayers: 4 >>> dis_num_heads: 4 >>> use_nll_oracle: 1 >>> use_nll_gen: 1 >>> use_nll_div: 1 >>> use_bleu: 1 >>> use_self_bleu: 0 >>> use_clas_acc: True >>> use_ppl: 0 >>> log_file: log/log_0413_0935_02.txt >>> save_root: save/20210413/oracle/sa_dpgan_pineapple_dt-Ra_lt-rsgan_mt-ra_et-Ra_sl20_temp1_lfd0.0_T0413_0935_02/ >>> signal_file: run_signal.txt >>> tips: DPGAN experiments |   |
|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---|